# **Image Dataset**

## Importação de pacotes e leitura dos dados

In [33]:
import numpy as np
import pandas as pd
import os
import imageio
from PIL import Image
import matplotlib.pyplot as plt
from utils import get_train_val_set_data
import tensorflow as tf


### Lendo as imagens em formato RGB

In [2]:
file_values = []
file_name = []
file_genre = []

folder_path = '../dataset/Data/images_rgb/'
genre_folders = os.listdir(folder_path)

for genre_folder in genre_folders:
  
  files = os.listdir(folder_path + genre_folder)
  print(genre_folder, len(files))
  
  for image in files:
    #im = imageio.imread(folder_path + genre_folder+ "/" +image)

    img = Image.open(folder_path + genre_folder+ "/" +image)
    image_array = np.array(img)
    file_values.append(image_array)
    #file_values.append(np.array(im).tolist())
    file_name.append(image)
    file_genre.append(genre_folder)

blues 100
classical 100
country 100
disco 100
hiphop 100
jazz 99
metal 100
pop 100
reggae 100
rock 100


In [41]:
df_dict = {
    'filename': file_name,
    'file_values': file_values,
    'label': file_genre,
}

df = pd.DataFrame(df_dict)

In [4]:
#df.file_values = df.file_values.apply(lambda x: x.tolist() )

In [5]:
df.head()

,filename,file_values,label
0,blues00000.png,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",blues
1,blues00001.png,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",blues
2,blues00002.png,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",blues
3,blues00003.png,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",blues
4,blues00004.png,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",blues


## Dividindo em treino, validação e teste

In [46]:
X_train, y_train, X_val, y_val, X_test, y_test = get_train_val_set_data(df)

-- Treino --
X (shape):  (699, 2)
y:  699
-- Validação --
X (shape):  (150, 2)
y:  150
-- Teste --
X (shape):  (150, 2)
y:  150


## Data Prep

### Features

In [47]:
X_train = np.concatenate(X_train.file_values.values).reshape(X_train.shape[0], 288, 432, 3)
X_val = np.concatenate(X_val.file_values.values).reshape(X_val.shape[0], 288, 432, 3)
X_test = np.concatenate(X_test.file_values.values).reshape(X_test.shape[0], 288, 432, 3)

### Variável target

Utilizando Label Encoding para transformar os generos musicais em números de 0 a 9

In [51]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

le.fit(y_train)

Y_train = le.transform(y_train)
Y_val = le.transform(y_val)
Y_Test = le.transform(y_test)

print('Exemplo antes:', Y_train[0])

Exemplo antes: 0


Transformando as saídas no formato de One Hot Encoding 

In [52]:
n_classes = 10

Y_train = tf.keras.utils.to_categorical(Y_train, n_classes)
Y_val = tf.keras.utils.to_categorical(Y_val, n_classes)
Y_test = tf.keras.utils.to_categorical(Y_Test, n_classes)

print('Exemplo depois:', Y_train[0])

Exemplo depois: [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


## Modelagens

In [53]:
input_shape = X_train[0].shape
input_shape

(288, 432, 3)

### VGG19 com camandas pré treinadas congeladas

In [54]:
vgg19_model = tf.keras.applications.VGG19(
    include_top=False,
    weights="imagenet",
    classes=1000,
    classifier_activation="softmax",
    input_shape = input_shape
)

for layer in vgg19_model.layers:
    layer.trainable = False

# Insere novas camadas no fim da rede para classificação
frozen_model = tf.keras.Sequential([
  vgg19_model,
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(10, activation='softmax')
])

frozen_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Functional)           (None, 9, 13, 512)        20024384  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 10)                5130      
Total params: 20,029,514
Trainable params: 5,130
Non-trainable params: 20,024,384
_________________________________________________________________


In [55]:
sgd = tf.keras.optimizers.SGD(learning_rate=0.001)

early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

#train_batches = datagen_resnet.flow(X_train, Y_train, shuffle=True, batch_size=64)
#val_batches = datagen_resnet.flow(X_val, Y_val, shuffle=True, batch_size=64)

In [59]:
frozen_model.compile(
    loss='categorical_crossentropy', 
    optimizer=sgd, metrics=['accuracy']
)



In [61]:
frozen_model.fit( 
            x = X_train, 
            y = Y_train, 
            validation_data = (X_val, Y_val), 
            epochs = 2, 
            batch_size = 10,
            callbacks = [early],
            shuffle = True
        )


Epoch 1/2
70/70 [==============================] - 1090s 16s/step - loss: 3.2336 - accuracy: 0.1731 - val_loss: 2.7476 - val_accuracy: 0.1733
Epoch 2/2
70/70 [==============================] - 1094s 16s/step - loss: 2.2151 - accuracy: 0.3033 - val_loss: 1.9443 - val_accuracy: 0.3467
